# Discover, Customize and Access NSIDC DAAC Data

This notebook is based off of the NSIDC-Data-Access-Notebook provided here: 
https://github.com/nsidc/NSIDC-Data-Access-Notebook

Now that we've visualized our study areas, we will explore data coverage, size, and customization service availability. We will then access the data we're interested in utilizing the NSIDC DAAC's Data Access and Service API. 

### Import packages


In [1]:
import requests, getpass, socket, json, zipfile, io, math, os, shutil, pprint, re, time
from statistics import mean
from requests.auth import HTTPBasicAuth

### Input Earthdata Login credentials

An Earthdata Login account is required to access data from the NSIDC DAAC. If you do not already have an Earthdata Login account, visit http://urs.earthdata.nasa.gov to register.

In [2]:
# Earthdata Login credentials

uid = input('Earthdata Login user name: ')
pswd = getpass.getpass('Earthdata Login password: ')
email = input('Email address associated with Earthdata Login account: ')

Earthdata Login user name:  amy.steiker
Earthdata Login password:  ·········
Email address associated with Earthdata Login account:  amy.steiker@nsidc.org


### Select data sets and determine version numbers

In [3]:
# Create dictionary of data set parameters we'll use in our access API command below. We'll start with data set IDs (e.g. ATL07) of interest here, also known as "short name".

data_dict = {
    'photon_height' : {'short_name' : 'ATL03'},
    'sea_ice_height' : {'short_name' : 'ATL07'},
    'ist' : {'short_name' : 'MOD29'},
    'bt' : {'short_name' : 'AU_SI6'}
}
pprint.pprint(data_dict)

{'bt': {'short_name': 'AU_SI6'},
 'ist': {'short_name': 'MOD29'},
 'photon_height': {'short_name': 'ATL03'},
 'sea_ice_height': {'short_name': 'ATL07'}}


In [4]:
# Get json response from CMR collection metadata

for i in range(len(data_dict)):
    cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
    response = requests.get(cmr_collections_url, params=list(data_dict.values())[i])
    results = json.loads(response.content) 

    # Find all instances of 'version_id' in metadata and print most recent version number
    versions = [el['version_id'] for el in results['feed']['entry']]
    versions = [i for i in versions if not any(c.isalpha() for c in i)]
    data_dict[list(data_dict.keys())[i]]['version'] = max(versions)
    print('The most recent version of ', list(data_dict.values())[i]['short_name'], ' is ', max(versions))

The most recent version of  ATL03  is  002
The most recent version of  ATL07  is  002
The most recent version of  MOD29  is  6
The most recent version of  AU_SI6  is  1


### Select time and area of interest

Data granules are returned based on a spatial bounding box and temporal range.

## Change following bbox and time code blocks - hardcode these into dictionary itself

In [5]:
# Bounding Box spatial parameter in 'W,S,E,N' format

# Input lower left longitude in decimal degrees
LL_lon = '140'
# Input lower left latitude in decimal degrees
LL_lat = '72'
# Input upper right longitude in decimal degrees
UR_lon = '153'
# Input upper right latitude in decimal degrees
UR_lat = '80'

bounding_box = LL_lon + ',' + LL_lat + ',' + UR_lon + ',' + UR_lat

#add bounding_box to each data set dictionary
for k, v in data_dict.items(): data_dict[k]['bounding_box'] = bounding_box

print(bounding_box)

140,72,153,80


In [6]:
#Input temporal range 

# Input start date in yyyy-MM-dd format
start_date = '2019-03-23'
# Input start time in HH:mm:ss format
start_time = '00:00:00'
# Input end date in yyyy-MM-dd format
end_date = '2019-03-23'
# Input end time in HH:mm:ss format
end_time = '23:59:59'

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'

#add temporal to each data set dictionary
for k, v in data_dict.items(): data_dict[k]['temporal'] = temporal

print(temporal)

2019-03-23T00:00:00Z,2019-03-23T23:59:59Z


### Determine how many granules exist over this time and area of interest, as well as the average size and total volume of those granules

In [7]:
# Query number of granules (paging over results)
granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
for i in range(len(data_dict)):
    params = {
        'short_name': list(data_dict.values())[i]['short_name'],
        'version': list(data_dict.values())[i]['version'],
        'bounding_box': bounding_box,
        'temporal': temporal,
        'page_size': 100,
        'page_num': 1
    }
    granules = []
    headers={'Accept': 'application/json'}
    while True:
        response = requests.get(granule_search_url, params=params, headers=headers)
        results = json.loads(response.content)

        if len(results['feed']['entry']) == 0:
            # Out of results, so break out of loop
            break

        # Collect results and increment page_num
        granules.extend(results['feed']['entry'])
        params['page_num'] += 1
    print('There are', len(granules), 'granules of', list(data_dict.values())[i]['short_name'], 'version', list(data_dict.values())[i]['version'], 'over my area and time of interest.')
    for k, v in data_dict.items(): data_dict[k]['gran_num'] = len(granules)
    granule_sizes = [float(granule['granule_size']) for granule in granules]
    print(f'The average size of each granule is {mean(granule_sizes):.2f} MB and the total size of all {len(granules)} granules is {sum(granule_sizes):.2f} MB')
    print()

There are 4 granules of ATL03 version 002 over my area and time of interest.
The average size of each granule is 1685.59 MB and the total size of all 4 granules is 6742.36 MB

There are 3 granules of ATL07 version 002 over my area and time of interest.
The average size of each granule is 260.65 MB and the total size of all 3 granules is 781.94 MB

There are 13 granules of MOD29 version 6 over my area and time of interest.
The average size of each granule is 2.75 MB and the total size of all 13 granules is 35.70 MB

There are 2 granules of AU_SI6 version 1 over my area and time of interest.
The average size of each granule is 88.35 MB and the total size of all 2 granules is 176.69 MB



Note that subsetting, reformatting, or reprojecting can alter the size of the granules if those services are applied to your request.

### Select the subsetting, reformatting, and reprojection services enabled for your data set of interest.

The NSIDC DAAC supports customization services on many of our NASA Earthdata mission collections. Let's discover whether or not our data set has these services available. If services are available, we will also determine the specific service options supported for this data set and select which of these services we want to request. 

In [8]:
from xml.etree import ElementTree as ET

for k, v in data_dict.items():
    sn = v['short_name']
    ve = (v['version'])
    capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{sn}.{ve}.xml'
    
    # Create session to store cookie and pass credentials to capabilities url
    session = requests.session()
    s = session.get(capability_url)
    response = session.get(s.url,auth=(uid,pswd))
    root = ET.fromstring(response.content)

    #collect lists with each service option
    subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

    # variable subsetting
    variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
    variables_raw = [variables[i]['value'] for i in range(len(variables))]
    variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
    variable_vals = [v.replace(':', '/') for v in variables_join]

    # reformatting
    formats = [Format.attrib for Format in root.iter('Format')]
    format_vals = [formats[i]['value'] for i in range(len(formats))]
    if format_vals : format_vals.remove('')

    # reprojection options
    projections = [Projection.attrib for Projection in root.iter('Projection')]
    proj_vals = []
    for i in range(len(projections)):
        if (projections[i]['value']) != 'NO_CHANGE' :
            proj_vals.append(projections[i]['value'])

    #print service information depending on service availability and select service options
    print(sn, 'service selection:')
    if len(subagent) < 1 :
            print('No services exist for', sn)
            meta = input('Would like to receive XML metadata files along with the science files? (y/n)')
            if meta == 'n': data_dict[k]['INCLUDE_META'] = 'N'
            print('')
    else:
        subdict = subagent[0]
        if subdict['spatialSubsetting'] == 'true':
            ss = input('Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n)')
            if ss == 'y': data_dict[k]['bbox'] = bounding_box
        if subdict['temporalSubsetting'] == 'true':
            ts = input('Subsetting by time, based on the temporal range inputted above, is available. Would you like to request this service? (y/n)')
            if ts == 'y': data_dict[k]['time'] = start_date + 'T' + start_time + ',' + end_date + 'T' + end_time 
        if len(format_vals) > 0 :
            print('These reformatting options are available:', format_vals)
            reformat = input('If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank.')
        if len(proj_vals) > 0 : 
            print('These reprojection options are available with your requested format:', proj_vals)
            data_dict[k]['projection'] = input('If you would like to reproject, copy and paste the reprojection option you would like (make sure to omit quotes, e.g. GEOGRAPHIC), otherwise leave blank.')
            # Enter required parameters for UTM North and South
            if data_dict[k]['projection'] == 'UTM NORTHERN HEMISPHERE' or data_dict[k]['projection'] == 'UTM SOUTHERN HEMISPHERE': 
                data_dict[k]['NZone'] = input('Please enter a UTM zone (1 to 60 for Northern Hemisphere; -60 to -1 for Southern Hemisphere):')
                data_dict[k]['projection_parameters'] = str('NZone:' + NZone)
        else: 
            print('No reprojection options are supported with your requested format')
    print()

ATL03 service selection:


Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n) y
Subsetting by time, based on the temporal range inputted above, is available. Would you like to request this service? (y/n) y


These reformatting options are available: ['TABULAR_ASCII', 'NetCDF4-CF', 'NetCDF-3']


If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank. 


No reprojection options are supported with your requested format

ATL07 service selection:


...

If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank. 


These reprojection options are available with your requested format: ['GEOGRAPHIC', 'UNIVERSAL TRANSVERSE MERCATOR', 'POLAR STEREOGRAPHIC', 'SINUSOIDAL']


If you would like to reproject, copy and paste the reprojection option you would like (make sure to omit quotes, e.g. GEOGRAPHIC), otherwise leave blank. POLAR STEREOGRAPHIC



AU_SI6 service selection:
No services exist for AU_SI6


Would like to receive XML metadata files along with the science files? (y/n) n


Now let's select a subset of variables. We'll use these primary variables of interest for the ICESat-2 sea ice and photon height products:


In [9]:
#ATL07
#Use only strong beams

data_dict['sea_ice_height']['coverage'] = '/gt1l/sea_ice_segments/delta_time,\
/gt1l/sea_ice_segments/latitude,\
/gt1l/sea_ice_segments/longitude,\
/gt1l/sea_ice_segments/heights/height_segment_confidence,\
/gt1l/sea_ice_segments/heights/height_segment_height,\
/gt1l/sea_ice_segments/heights/height_segment_quality,\
/gt1l/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt2l/sea_ice_segments/delta_time,\
/gt2l/sea_ice_segments/latitude,\
/gt2l/sea_ice_segments/longitude,\
/gt2l/sea_ice_segments/heights/height_segment_confidence,\
/gt2l/sea_ice_segments/heights/height_segment_height,\
/gt2l/sea_ice_segments/heights/height_segment_quality,\
/gt2l/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt3l/sea_ice_segments/delta_time,\
/gt3l/sea_ice_segments/latitude,\
/gt3l/sea_ice_segments/longitude,\
/gt3l/sea_ice_segments/heights/height_segment_confidence,\
/gt3l/sea_ice_segments/heights/height_segment_height,\
/gt3l/sea_ice_segments/heights/height_segment_quality,\
/gt3l/sea_ice_segments/heights/height_segment_surface_error_est'

#ATL03
#Use only strong beams

data_dict['photon_height']['coverage'] = '/ds_surf_type,\
/gt1l/heights/delta_time,\
/gt1l/heights/h_ph,\
/gt1l/heights/lat_ph,\
/gt1l/heights/lon_ph,\
/gt1l/heights/signal_conf_ph,\
/gt2l/heights/delta_time,\
/gt2l/heights/h_ph,\
/gt2l/heights/lat_ph,\
/gt2l/heights/lon_ph,\
/gt2l/heights/signal_conf_ph,\
/gt3l/heights/delta_time,\
/gt3l/heights/h_ph,\
/gt3l/heights/lat_ph,\
/gt3l/heights/lon_ph,\
/gt3l/heights/signal_conf_ph'

### Select data access configurations

The data request can be accessed asynchronously or synchronously. The asynchronous option will allow concurrent requests to be queued and processed without the need for a continuous connection. Those requested orders will be delivered to the specified email address, or they can be accessed programmatically as shown below. Synchronous requests will automatically download the data as soon as processing is complete. For this tutorial, we will be selecting the asynchronous method. 

In [10]:
#Set NSIDC data access base URL
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'

for k, v in data_dict.items():
    #Add email address
    data_dict[k]['email'] = email
    
    #Set the request mode to asynchronous
    data_dict[k]['request_mode'] = 'async'

    #Set the page size to the maximum for asynchronous requests 
    page_size = 2000
    data_dict[k]['page_size'] = page_size

    #Determine number of orders needed for requests over 2000 granules. 
    page_num = math.ceil(data_dict[k]['gran_num']/page_size)
    data_dict[k]['page_num'] = page_num
    del data_dict[k]['gran_num']
    print('There will be', page_num, 'total order(s) processed for our', v['short_name'], 'request.')

There will be 1 total order(s) processed for our ATL03 request.
There will be 1 total order(s) processed for our ATL07 request.
There will be 1 total order(s) processed for our MOD29 request.
There will be 1 total order(s) processed for our AU_SI6 request.


### Create the API endpoint 

Programmatic API requests are formatted as HTTPS URLs that contain key-value-pairs specifying the service operations that we specified above. The following command can be executed via command line, a web browser, or in Python below. 

In [11]:
endpoint_list = [] 
for k, v in data_dict.items():
    param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in v.items())
    param_string = param_string.replace("'","")
    
    #Print API base URL + request parameters
    API_request = api_request = f'{base_url}?{param_string}'
    endpoint_list.append(API_request)
    if data_dict[k]['page_num'] > 1:
        for i in range(data_dict[k]['page_num']):
            page_val = i + 2
            data_dict[k]['page_num'] = page_val
            API_request = api_request = f'{base_url}?{param_string}'
            endpoint_list.append(API_request)

print("\n".join("\n"+s for s in endpoint_list))


https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL03&version=002&bounding_box=140,72,153,80&temporal=2019-03-23T00:00:00Z,2019-03-23T23:59:59Z&bbox=140,72,153,80&time=2019-03-23T00:00:00,2019-03-23T23:59:59&coverage=/ds_surf_type,/gt1l/heights/delta_time,/gt1l/heights/h_ph,/gt1l/heights/lat_ph,/gt1l/heights/lon_ph,/gt1l/heights/signal_conf_ph,/gt2l/heights/delta_time,/gt2l/heights/h_ph,/gt2l/heights/lat_ph,/gt2l/heights/lon_ph,/gt2l/heights/signal_conf_ph,/gt3l/heights/delta_time,/gt3l/heights/h_ph,/gt3l/heights/lat_ph,/gt3l/heights/lon_ph,/gt3l/heights/signal_conf_ph&email=amy.steiker@nsidc.org&request_mode=async&page_size=2000&page_num=1

https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL07&version=002&bounding_box=140,72,153,80&temporal=2019-03-23T00:00:00Z,2019-03-23T23:59:59Z&bbox=140,72,153,80&time=2019-03-23T00:00:00,2019-03-23T23:59:59&coverage=/gt1l/sea_ice_segments/delta_time,/gt1l/sea_ice_segments/latitude,/gt1l/sea_ice_segments/longitude,/gt1l/sea_ice_segment

### Request data

We will now download data using the Python requests library. The data will be downloaded directly to this notebook directory in a new Outputs folder. The progress of each order will be reported.

In [12]:
# Create an output folder if the folder does not already exist.
path = str(os.getcwd() + '/Outputs')
if not os.path.exists(path):
    os.mkdir(path)

# Request data service for each page number, and unzip outputs
for k, v in data_dict.items():
    for i in range(data_dict[k]['page_num']):
        page_val = i + 1
        print(v['short_name'], 'Order: ', page_val)

    # For all requests other than spatial file upload, use get function
        request = session.get(base_url, params=v.items())
        print('Request HTTP response: ', request.status_code)

    # Raise bad request: Loop will stop for bad response code.
        request.raise_for_status()
        #print('Order request URL: ', request.url)
        esir_root = ET.fromstring(request.content)
        #print('Order request response XML content: ', request.content)

    #Look up order ID
        orderlist = []   
        for order in esir_root.findall("./order/"):
            orderlist.append(order.text)
        orderID = orderlist[0]
        print('order ID: ', orderID)

    #Create status URL
        statusURL = base_url + '/' + orderID
        print('status URL: ', statusURL)

    #Find order status
        request_response = session.get(statusURL)    
        print('HTTP response from order response URL: ', request_response.status_code)

    # Raise bad request: Loop will stop for bad response code.
        request_response.raise_for_status()
        request_root = ET.fromstring(request_response.content)
        statuslist = []
        for status in request_root.findall("./requestStatus/"):
            statuslist.append(status.text)
        status = statuslist[0]
        print('Data request ', page_val, ' is submitting...')
        print('Initial request status is ', status)

    #Continue loop while request is still processing
        while status == 'pending' or status == 'processing': 
            print('Status is not complete. Trying again.')
            time.sleep(10)
            loop_response = session.get(statusURL)

    # Raise bad request: Loop will stop for bad response code.
            loop_response.raise_for_status()
            loop_root = ET.fromstring(loop_response.content)

    #find status
            statuslist = []
            for status in loop_root.findall("./requestStatus/"):
                statuslist.append(status.text)
            status = statuslist[0]
            print('Retry request status is: ', status)
            if status == 'pending' or status == 'processing':
                continue

    #Order can either complete, complete_with_errors, or fail:
    # Provide complete_with_errors error message:
        if status == 'complete_with_errors' or status == 'failed':
            messagelist = []
            for message in loop_root.findall("./processInfo/"):
                messagelist.append(message.text)
            print('error messages:')
            pprint.pprint(messagelist)

    # Download zipped order if status is complete or complete_with_errors
        if status == 'complete' or status == 'complete_with_errors':
            downloadURL = 'https://n5eil02u.ecs.nsidc.org/esir/' + orderID + '.zip'
            print('Zip download URL: ', downloadURL)
            print('Beginning download of zipped output...')
            zip_response = session.get(downloadURL)
            # Raise bad request: Loop will stop for bad response code.
            zip_response.raise_for_status()
            with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
                z.extractall(path)
            print('Data request', page_val, 'is complete.')
        else: print('Request failed.')
    print()

ATL03 Order:  1
Request HTTP response:  201
order ID:  5000000412850
status URL:  https://n5eil02u.ecs.nsidc.org/egi/request/5000000412850
HTTP response from order response URL:  201
Data request  1  is submitting...
Initial request status is  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  complete
Zip download URL:  https://n5eil02u.ecs.nsidc.org/esir/5000000412850.zip
Beginning download of zipped output...
Data request 1 is complete.

ATL07 

IndexError: list index out of range

In [18]:
print(request.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<eesi:agentResponse xsi:schemaLocation="http://eosdis.nasa.gov/esi/rsp/e https://newsroom.gsfc.nasa.gov/esi/8.1/schemas/ESIAgentResponseExternal.xsd" xmlns="" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n    <downloadUrls>\n        <downloadUrl>ftp://n5eil02u.ecs.nasa.gov/DP1/AMSA/AU_SI6.001/2019.03.22/AMSR_U2_L3_SeaIce6km_B02_20190322.he5</downloadUrl>\n        <downloadUrl>ftp://n5eil02u.ecs.nasa.gov/DP1/AMSA/AU_SI6.001/2019.03.23/AMSR_U2_L3_SeaIce6km_B02_20190323.he5</downloadUrl>\n    </downloadUrls>\n</eesi:agentResponse>\n'


### Finally, we will clean up the Output folder by removing individual order folders:

In [19]:
# Clean up Outputs folder by removing individual granule folders 

for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
    for name in dirs:
        os.rmdir(os.path.join(root, name))    

### To review, we have explored data availability and volume over a region and time of interest, discovered and selected data customization options, constructed API endpoints for our requests, and downloaded data. Let's move on to the analysis portion of the tutorial.